# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

### Шаг 2.1 Заполнение пропусков

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 11 столбцов с разными типами данных: "int64", "float64", "object"

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В двух столбцах "days_employed" и "total_income" существуют пропуски данных, а именно "общий трудовой стаж в днях" и "ежемесячный доход" соответственно.

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
data.isna().sum() #подсчет пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
percentage = len(data[data['total_income'].isna()]) / len(data['children'])
print(f'Доля пропусков {percentage:.1%}')

Доля пропусков 10.1%


In [7]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

 **Замена пропусков**

In [10]:
grp = data.groupby('income_type')['total_income'].median()
grp

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [11]:
grp = data.groupby(['income_type'])
data['total_income'] = grp.total_income.apply(lambda x: x.fillna(x.median()))
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [12]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [13]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Пропуски в столбце `total_income` были заменены на медианное значение, так как оно обладает более объективной информацией о зарплате нежели среднее значение по данным параметрам.

Пропуски в стоблце могли появиться из-за разных причин:
* Не правильно введенные данные;
* Не полностью заполненные данные клиентами сервиса;
* Сбой системы.

### Шаг 2.2 Проверка данных на аномалии и исправления.

**Замена в столбце** `children`

In [15]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [16]:
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20, 2)

In [17]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Замена в стоблце** `days_employed`

In [18]:
data['days_employed'] = abs(data['days_employed'])
median_data = data['days_employed'].median()
columns_to_replace = ['days_employed']
for columns in columns_to_replace:
    data['days_employed'] = data['days_employed'].fillna(median_data)
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [19]:
data['days_employed'].describe()

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

В данном столбце есть значения, которые явно являются ошибочные, к примеру: максимальный стаж равен 401755 дней, что ровняется ~ 1100 лет, что свидетельствует об ошибке при вводе данных в таблицу. Данный столбец не участвует в процессе анализа данных по данной таблице. Но в дальнейшем, стоило бы уточнить у человека, предоставившего эти данные, откуда взялись даннные цифры. (P.S. а вдруг у них есть люди, которые так долго у них работают..(бедные люди))

In [20]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Замена в столбце** `dob_years`

In [21]:
median_age = data['dob_years'].median()

data['dob_years'] = data['dob_years'].replace(0, median_age)

In [22]:
data['dob_years'].describe()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

**Замена в столбце** `gender`

In [23]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [24]:
data[data['gender'].isin(['XNA'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [25]:
data['gender'] = data['gender'].drop(labels = [10701],axis = 0)

In [26]:
data[data['gender'].isin(['XNA'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Мы можем удалить одну строку датафрейма так как это не отобразиться существенно на общем анализе данных

### Шаг 2.3. Изменение типов данных.

In [27]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data = data.rename(columns={'dob_years':'age'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   age               21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

`children`

In [28]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

`family_status`

In [29]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

`education`

In [30]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [31]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

`income_type`

**Замена в столбце** `days_employed`

In [32]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [33]:
data['days_employed'].describe()

count     21525.000000
mean      60377.561580
std      133257.545602
min          24.000000
25%        1025.000000
50%        2194.000000
75%        4779.000000
max      401755.000000
Name: days_employed, dtype: float64

`family_status_id`

В данном столбце есть значения, которые явно являются ошибочные, к примеру: максимальный стаж равен 401755 дней, что ровняется ~ 1100, что свидетельствует об ошибке при вводе данных в таблицу. Данный столбец не участвует в процессе анализа данных по данной таблице. Но в дальнейшем, стоило бы уточнить у человека, предоставившего эти данные, откуда взялись даннные цифры. (P.S. а вдруг у них есть люди которые так долго у них работают..(бедные люди))

In [34]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

`debt`

In [35]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

`total_income`

In [36]:
data['total_income'].value_counts()

142594    1105
172357     509
118514     414
150447     147
144533       3
          ... 
179138       1
61724        1
99269        1
151371       1
126820       1
Name: total_income, Length: 18608, dtype: int64

In [37]:
data.duplicated().sum()

72

In [38]:
data = data.drop_duplicates().reset_index(drop=True)

In [39]:
data.duplicated().sum()

0

Сначала методом `drop_duplicates().reset_index(drop=True)` удалили все повторяющиеся строки

Далее выявил все неявные дупликаты методом `value_counts`. В столбец `education` были найдены одни и те же значени, записанные в разном регистре и методом `str.lower()` все было приведено к одному регистру.

Также были проверены остальные столбцы в которых могли бы быть неявные дубликаты.

Дубликаты могли появиться по абсолютно разным причинам:
* Сбой системы;
* Ошибка специалиста, который предоставил данные;
* Если таблица заполнялась людьми лично, то клиент банка из-за разрыва соединения, или просто случайно нажать несколько раз на "ок" при введение данных на сайте;
* Пропуски значений могли появитья по причине сбоя системы или нежелания клиента указать определенные данные (к примеру: опосаясь, что именно эти данные повлияют на решение банка о выдаче кредита)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [40]:
education = data[['education', 'education_id']]
education = education.drop_duplicates().reset_index(drop=True)
education.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [41]:
family_status = data[['family_status', 'family_status_id']]
family_status = family_status.drop_duplicates().reset_index(drop=True)
family_status.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [42]:
data = data.drop(['education', 'family_status'], axis=1)
data.head(10)

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

`total_income_category`

In [43]:
def total_income_category(income):
    
    
    #0–30000 — 'E'; 
    #30001–50000 — 'D'; 
    #50001–200000 — 'C';
    #200001–1000000 — 'B';
    #1000001 и выше — 'A'.
    
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    if income > 1000000:
        return 'A'

In [44]:
data['total_income_category'] = data['total_income'].apply(total_income_category)

In [45]:
data.head(10)

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [46]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [47]:
def purpose_category(purpose):
    
    #'операции с автомобилем',
    #'операции с недвижимостью',
    #'проведение свадьбы',
    #'получение образования'
    
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    

data['purpose_category'] = data['purpose'].apply(purpose_category)

In [48]:
data['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [53]:
data_pivot1 = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot1

,count,sum,mean
,debt,debt,debt
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Изучив данные сводной таблицы количестве детей и факте наличия задолжностей по кредитам, можно сделать определенные выводы:
* Люди, имеющие 4 ребенка, чаще всего не возвращают кредит;
* Люди, имеющие 5 детей, не имеют задолжностей по кредитам;

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [54]:
data_pivot2 = data.pivot_table(index='family_status_id', values='debt', aggfunc=['count', 'sum', 'mean'])

In [55]:
data_pivot2 = family_status.merge(data_pivot2, on='family_status_id')

/opt/conda/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [56]:
data_pivot2

,family_status,family_status_id,"(count, debt)","(sum, debt)","(mean, debt)"
0,женат / замужем,0,12339,931,0.075452
1,гражданский брак,1,4150,388,0.093494
2,вдовец / вдова,2,959,63,0.065693
3,в разводе,3,1195,85,0.071130
4,Не женат / не замужем,4,2810,274,0.097509


Изучив данные сводной таблицы, можно сделать определенные выводы
* Доля не женатых/не замужем людей, которые имеют задолжности по кредитам больше всего;
* Вдовы / вдовцы имеют меньше всего задолжностей по кредитам.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
data_pivot3 = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])

In [58]:
data_pivot3

    #0–30000 — 'E'; 
    #30001–50000 — 'D'; 
    #50001–200000 — 'C';
    #200001–1000000 — 'B';
    #1000001 и выше — 'A'.

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5042,356,0.070607
C,16014,1360,0.084926
D,350,21,0.060000
E,22,2,0.090909


Изучив данные сводной таблицы и гистаграммы, можно сделать определенные выводы:
* Люди, имеющие доходы "0–30000", имеют больше всего задолжностей по кредитам;
* Люди, имеющие доходы "30001–50000", имеют меньше всего задолжностей по кредитам.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [59]:
data_pivot4 = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])

In [60]:
data_pivot4

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2323,186,0.080069


Изучив данные сводной таблицы, можно сделать определенные выводы:
* Больше всего задолжностей по кредитам у людей, которые берут кредит для операций с автомобилем;
* Меньше всего задолжностей по кредитам у людей, которые берут кредит для операций с недвижимостью.

## Общий вывод:

Проанализорав данные статистики о платёжеспособности клиентов, проверили зависимости некоторых критериев, влияющих на возврат кредитов в срок.

1. Зависимость между количеством детей и возвратом кредита в срок:
* Люди, имеющие 4 ребенка, чаще всего не возвращают кредит;
* Люди, имеющие 5 детей, не имеют задолжностей по кредитам;

2. Зависимость между семейным положением и возвратом кредита в срок:
* Доля не женатых/не замужем людей, которые имеют задолжности по кредитам больше всего;
* Вдовы / вдовцы имеют меньше всего задолжностей по кредитам.

3. Зависимость между уровнем дохода и возвратом кредита в срок:
* Люди, имеющие доходы "0–30000", имеют больше всего задолжностей по кредитам;
* Люди, имеющие доходы "30001–50000", имеют меньше всего задолжностей по кредитам.

4. Как разные цели кредита влияют на его возврат в срок:
* Больше всего задолжностей по кредитам у людей, которые берут кредит для операций с автомобилем;
* Меньше всего задолжностей по кредитам у людей, которые берут кредит для операций с недвижимостью.